# Exercise 1

Import libraries.

In [ ]:
import pandas as pd

Scrape the Wikipedia page and get the content of the first table.

In [14]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url)[0]
df.head(3)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods


Remove cells that do not have an assigned borough.

In [224]:
df = df[df.Borough != 'Not assigned']
df.reset_index(drop=True, inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752450,-79.329910
1,M4A,North York,Victoria Village,43.730570,-79.313060
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655120,-79.362640
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723570,-79.437110
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.660673,-79.390835


There is no Borough with a 'Not assigned' neighbourhood and no neighbourhood is listed twice.

In [21]:
df.shape

(103, 3)

# Exercise 2

Import libraries.

In [24]:
import geocoder

In [149]:
long_list = list()
lat_list = list()
for pc, br, nh in zip(df['Postal Code'].to_list(), df['Borough'].to_list(), df['Neighbourhood'].to_list()):
    g = geocoder.arcgis(('{}, {}, {}').format(pc, br, nh))[0]
    long, lat = g.geometry['coordinates']
    long_list.append(long)
    lat_list.append(lat)

df['Latitude'] = lat_list
df['Longitude'] = long_list
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752450,-79.329910
1,M4A,North York,Victoria Village,43.730570,-79.313060
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655120,-79.362640
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723570,-79.437110
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.660673,-79.390835


# Exercise 3

Import libraries.

In [232]:
import requests
import numpy as np
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

First drop rows, which do not contain 'Toronto' in their borough names. 

In [223]:
toronto = df[df['Borough'].str.contains('Toronto')]
toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655120,-79.362640
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.660673,-79.390835
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657390,-79.378040
3,M5C,Downtown Toronto,St. James Town,43.671100,-79.373590
4,M4E,East Toronto,The Beaches,43.674130,-79.296440


Foursquare API credentials and API version. 

In [158]:
CLIENT_ID = '' # Foursquare client id
CLIENT_SECRET = '' # Foursquare client secret
VERSION = '20180605' # Foursquare API version

Function to extract nearby values into a pandas DataFrame.

In [163]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return nearby_venues

Get the ten nearest venues for each neighborhood.

In [164]:
LIMIT = 10
venues = getNearbyVenues(
    toronto['Neighbourhood'].to_list(), 
    toronto['Latitude'].to_list(), 
    toronto['Longitude'].to_list())

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

Let's check how many venues were returned for each neighborhood.

In [167]:
toronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,10,10,10,10,10,10
"Brockton, Parkdale Village, Exhibition Place",10,10,10,10,10,10
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",10,10,10,10,10,10
Central Bay Street,10,10,10,10,10,10
Christie,10,10,10,10,10,10


Let's check how many unique venue categories exist.

In [171]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 119 uniques categories.


One hot encode the venue column.

In [186]:
toronto_venues_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

Drop 'Neighborhood' category as it is not really a venue.

In [221]:
toronto_venues_onehot.drop('Neighborhood' ,axis=1, inplace=True)

Add Neighborhood column to the DataFrame and move it to the first column.

In [222]:
toronto_venues_onehot.insert(0, 'Neighborhood', toronto_venues['Neighborhood'])
toronto_venues_onehot.head()

,Neighborhood,American Restaurant,Antique Shop,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [200]:
toronto_grouped = toronto_venues_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,American Restaurant,Antique Shop,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Christie,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Print each neighborhood along with the top 5 most common venues.

In [202]:
num_top_venues = 5

for nh in toronto_grouped['Neighborhood']:
    print("----"+nh+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == nh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                           venue  freq
0                   Cocktail Bar   0.1
1                           Park   0.1
2  Vegetarian / Vegan Restaurant   0.1
3                      Gastropub   0.1
4                 Farmers Market   0.1


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0             Coffee Shop   0.2
1  Furniture / Home Store   0.1
2      Italian Restaurant   0.1
3       French Restaurant   0.1
4            Cocktail Bar   0.1


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0                Diner   0.1
1    Food & Drink Shop   0.1
2         Optical Shop   0.1
3  Peruvian Restaurant   0.1
4   Seafood Restaurant   0.1


----Central Bay Street----
                venue  freq
0  Seafood Restaurant   0.1
1       Shopping Mall   0.1
2      Clothing Store   0.1
3     Bubble Tea Shop   0.1
4           Gastropub   0.1


----Chris

Store the data in a pandas DataFrame.

In [207]:
# function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [208]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Food Truck,Gastropub,Park,Cocktail Bar,Restaurant,Farmers Market,Museum,Fountain,Creperie,Vegetarian / Vegan Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Hotel,Bakery,Cocktail Bar,French Restaurant,Furniture / Home Store,Breakfast Spot,Gym,Italian Restaurant,Grocery Store
2,"CN Tower, King and Spadina, Railway Lands, Har...",Diner,Beer Bar,Dessert Shop,Peruvian Restaurant,Gym,Spanish Restaurant,Coffee Shop,Optical Shop,Food & Drink Shop,Seafood Restaurant
3,Central Bay Street,Comic Shop,Clothing Store,Modern European Restaurant,Shopping Mall,Seafood Restaurant,Sushi Restaurant,Miscellaneous Shop,Bubble Tea Shop,Gastropub,Event Space
4,Christie,Grocery Store,American Restaurant,Gym,Candy Store,Coffee Shop,Baby Store,Park,Italian Restaurant,Café,Event Space


In [209]:
toronto_grouped.head()

,Neighborhood,American Restaurant,Antique Shop,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Christie,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Clustering

K-Means clustering.

In [238]:
# set number of clusters
k = int(3)

X_train = toronto_grouped.drop('Neighborhood', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=4).fit(X_train)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([0, 2, 0, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 0, 1, 0, 1, 2, 1, 0, 2, 2,
       2, 1, 1, 2, 1, 1, 0, 1, 0, 2, 1, 0, 1, 0])

Create new dataframe, which includes the label as well as the top 10 venues for each neighborhood.

In [243]:
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# Join toronto_grouped with toronto to add latitude/longitude for each neighborhood
toronto_join = toronto.join(toronto_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
# drop rows with missing values
toronto_join.dropna(inplace=True)

toronto_join.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655120,-79.362640,1.0,Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Pub,Event Space,Bakery,Spa,Gay Bar,Flea Market
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.660673,-79.390835,2.0,Coffee Shop,Spa,Fried Chicken Joint,Gastropub,Park,Italian Restaurant,Portuguese Restaurant,Dog Run,Donut Shop,Food & Drink Shop
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657390,-79.378040,0.0,Diner,Theater,Café,Music Venue,Clothing Store,Ramen Restaurant,Burrito Place,Comic Shop,Plaza,Pizza Place
3,M5C,Downtown Toronto,St. James Town,43.671100,-79.373590,0.0,Grocery Store,Pie Shop,Bar,Caribbean Restaurant,Diner,Bistro,Library,Market,Filipino Restaurant,Flea Market
4,M4E,East Toronto,The Beaches,43.674130,-79.296440,0.0,Nail Salon,Bar,Mediterranean Restaurant,Breakfast Spot,Trail,Toy / Game Store,Japanese Restaurant,Tea Room,Chocolate Shop,Juice Bar


Visualize the resulting clusters.

In [230]:
address = 'Toronto, Ontario, Kanada'
longitude, latitude = geocoder.arcgis(address)[0].geometry['coordinates']

print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.648690000000045, -79.38543999999996.


In [266]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_join['Latitude'], 
                                  toronto_join['Longitude'],
                                  toronto_join['Neighbourhood'], 
                                  toronto_join['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Let's have a look at the clusters.

**Label = 2**

In [268]:
toronto_join.loc[toronto_join['Cluster Labels'] == 2, toronto_join.columns[[1] + list(range(5, toronto_join.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,2.0,Coffee Shop,Spa,Fried Chicken Joint,Gastropub,Park,Italian Restaurant,Portuguese Restaurant,Dog Run,Donut Shop,Food & Drink Shop
14,West Toronto,2.0,Coffee Shop,Hotel,Bakery,Cocktail Bar,French Restaurant,Furniture / Home Store,Breakfast Spot,Gym,Italian Restaurant,Grocery Store
18,Central Toronto,2.0,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bubble Tea Shop,Garden,Sushi Restaurant,Bank,BBQ Joint,Farm,French Restaurant
20,Central Toronto,2.0,Post Office,Italian Restaurant,Yoga Studio,Fountain,Diner,Dog Run,Donut Shop,Eastern European Restaurant,Event Space,Farm
23,Central Toronto,2.0,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bubble Tea Shop,Garden,Sushi Restaurant,Bank,BBQ Joint,Farm,French Restaurant
25,West Toronto,2.0,Coffee Shop,Park,Ice Cream Shop,Juice Bar,Café,Sushi Restaurant,Bakery,Eastern European Restaurant,Food Truck,Event Space
31,Central Toronto,2.0,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Food & Drink Shop,Flea Market,Filipino Restaurant,Farmers Market,Farm,Yoga Studio
33,Downtown Toronto,2.0,Italian Restaurant,Yoga Studio,BBQ Joint,Salon / Barbershop,Bar,Juice Bar,Coffee Shop,Park,Event Space,Fountain


**Label = 1**

In [270]:
toronto_join.loc[toronto_join['Cluster Labels'] == 1, toronto_join.columns[[1] + list(range(5, toronto_join.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1.0,Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Pub,Event Space,Bakery,Spa,Gay Bar,Flea Market
7,Downtown Toronto,1.0,Grocery Store,American Restaurant,Gym,Candy Store,Coffee Shop,Baby Store,Park,Italian Restaurant,Café,Event Space
8,Downtown Toronto,1.0,Restaurant,American Restaurant,Coffee Shop,Seafood Restaurant,Café,Steakhouse,Bakery,Gym / Fitness Center,Vegetarian / Vegan Restaurant,Donut Shop
9,West Toronto,1.0,Café,Cupcake Shop,Comedy Club,Music Store,South American Restaurant,Coffee Shop,Cocktail Bar,Vegetarian / Vegan Restaurant,Eastern European Restaurant,Donut Shop
11,West Toronto,1.0,Restaurant,Café,Japanese Restaurant,Boutique,Thai Restaurant,Coffee Shop,Wine Bar,Eastern European Restaurant,Event Space,Donut Shop
13,Downtown Toronto,1.0,Coffee Shop,Café,Hotel,Pub,Restaurant,Bakery,Gym,Eastern European Restaurant,Food Truck,Donut Shop
16,Downtown Toronto,1.0,Café,Pub,Restaurant,Gastropub,Bakery,American Restaurant,Gym,Gym / Fitness Center,Coffee Shop,Eastern European Restaurant
19,Central Toronto,1.0,Pharmacy,Garden,Café,Skating Rink,Yoga Studio,Food Truck,Diner,Dog Run,Donut Shop,Eastern European Restaurant
22,West Toronto,1.0,Yoga Studio,Gastropub,Antique Shop,Arts & Crafts Store,Seafood Restaurant,Italian Restaurant,Flea Market,Organic Grocery,Café,Bar
24,Central Toronto,1.0,History Museum,Mexican Restaurant,Donut Shop,Pub,Café,Burger Joint,Middle Eastern Restaurant,Historic Site,Indian Restaurant,BBQ Joint


**Label = 0**

In [271]:
toronto_join.loc[toronto_join['Cluster Labels'] == 0, toronto_join.columns[[1] + list(range(5, toronto_join.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0.0,Diner,Theater,Café,Music Venue,Clothing Store,Ramen Restaurant,Burrito Place,Comic Shop,Plaza,Pizza Place
3,Downtown Toronto,0.0,Grocery Store,Pie Shop,Bar,Caribbean Restaurant,Diner,Bistro,Library,Market,Filipino Restaurant,Flea Market
4,East Toronto,0.0,Nail Salon,Bar,Mediterranean Restaurant,Breakfast Spot,Trail,Toy / Game Store,Japanese Restaurant,Tea Room,Chocolate Shop,Juice Bar
5,Downtown Toronto,0.0,Food Truck,Gastropub,Park,Cocktail Bar,Restaurant,Farmers Market,Museum,Fountain,Creperie,Vegetarian / Vegan Restaurant
6,Downtown Toronto,0.0,Comic Shop,Clothing Store,Modern European Restaurant,Shopping Mall,Seafood Restaurant,Sushi Restaurant,Miscellaneous Shop,Bubble Tea Shop,Gastropub,Event Space
10,Downtown Toronto,0.0,Plaza,Supermarket,Coffee Shop,Salad Place,Basketball Stadium,Sporting Goods Shop,Lounge,Hotel,IT Services,Train Station
15,East Toronto,0.0,Nail Salon,Bar,Mediterranean Restaurant,Breakfast Spot,Trail,Toy / Game Store,Japanese Restaurant,Tea Room,Chocolate Shop,Juice Bar
21,Central Toronto,0.0,Park,Sushi Restaurant,Gym / Fitness Center,Trail,Jewelry Store,Flea Market,Filipino Restaurant,Farmers Market,Farm,Event Space
26,Central Toronto,0.0,Dessert Shop,Diner,Gym,Indian Restaurant,Italian Restaurant,Park,Thai Restaurant,Sandwich Place,Gas Station,Restaurant
27,Downtown Toronto,0.0,Yoga Studio,Bakery,Dessert Shop,Restaurant,Beer Bar,Bar,Sushi Restaurant,Japanese Restaurant,Italian Restaurant,French Restaurant
